In [1]:
# Primary imports and constants
import ROOT
import numpy as np
import matplotlib.pyplot as plot
import datetime
import pandas as pd
import os
import glob
import math
import io
from PIL import Image, ImageOps
import iy10_ultrashower_utilities as shower
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz 
px = 1/plot.rcParams['figure.dpi']  # pixel in inches

# Functions
def ProcessTimestamp( evtTimestamp ):
    evtNanosec = evtTimestamp & 0xFFFFFFFF
    evtTimestamp = (evtTimestamp >> 32) & 0xFFFFFFFF
    evtTime = datetime.datetime.fromtimestamp(evtTimestamp + evtNanosec*1e-9)
    return evtTime

# Custom rectangular markers
from matplotlib.path import Path
verts = [
   (0., 0.),  # left, bottom
   (0., 3.9),  # left, top
   (6, 3.9),  # right, top
   (6, 0.),  # right, bottom
   (0., 0.),  # back to left, bottom
]
codes = [
    Path.MOVETO, #begin drawing
    Path.LINETO, #straight line
    Path.LINETO,
    Path.LINETO,
    Path.CLOSEPOLY, #close shape. This is not required for this shape but is "good form"
]
recMarker = Path(verts, codes)

#Reading subrun_tree and event_tree

file_path = glob.glob("/home/dwwhitti/nova/airshower/N23-07-05/*.root")
print(len(file_path))
for n in range(0,51): #Run this batchwise because the kernel dies after 58 files, create a self controlled loop that runs a batch of files each time
    base_name = os.path.basename(file_path[n])
    #print(base_name)
    name_split = os.path.splitext(base_name)
    #print(name_split)
    file_base_name = name_split[0]
    print(file_base_name)
    inFile = ROOT.TFile.Open(file_path[n],"READ")
    eventTree = inFile.Get("ultrashowerimage/Events")
    subrunTree = inFile.Get("ultrashowerimage/SubrunInfo")
    AsymXZ = []
    AsymYZ = []
    occupancyXZ = []
    occupancyYZ = []
    SubrunInfo = []
    Subrun_start = []
    Subrun_stop = []
    Subrun_number = []
    Run = []
    Subrun = []
    Event = []
    S_start = []
    S_stop = []
    AngleXZ = []
    AngleYZ = []
    Evt_timestamp = []
    CombinedList = []

    for index in range(subrunTree.GetEntries()):
        subrunTree.GetEntry(index)
        run = getattr(subrunTree, "run")
        subrun = getattr(subrunTree, "subrun")
        Subrun_number.append(subrun)
        srStart = getattr(subrunTree,"srStart")
        Subrun_start.append(srStart)
        srStop = getattr(subrunTree, "srStop")
        Subrun_stop.append(srStop)
        SubrunInfo.append((run,subrun,srStart,srStop))

    np.array(SubrunInfo)
    header_sinfo = ['Run','Subrun','Subrun_start','Subrun_stop']
    file_name = f"{file_base_name}_SubrunInfo.txt"
    np.savetxt(file_name, SubrunInfo, delimiter='\t', fmt='%d', header='\t'.join(header_sinfo), comments='#')

    for index in range(eventTree.GetEntries()):
        eventTree.GetEntry(index)
        hitsXZ = getattr(eventTree,"hitsXZ")
        hitsYZ = getattr(eventTree,"hitsYZ")
        run = getattr(eventTree,"run")
        subrun = getattr(eventTree,"subrun")
        event = getattr(eventTree,"event")
        timestamp = getattr(eventTree,"timestamp")
        
        unique_subruns = set()
        for i in range(len(Subrun_number)):
            if Subrun_number[i]==subrun and Subrun_number[i] not in unique_subruns:
                unique_subruns.add(Subrun_number[i])
                S_start.append(Subrun_start[i])
                S_stop.append(Subrun_stop[i])
            else:
                continue
        
        Run.append(run)
        Subrun.append(subrun)
        Event.append(event)
        
        arrayXZ = np.stack( ( hitsXZ.GetX(), hitsXZ.GetY() ), axis=1 )
        arrayYZ = np.stack( ( hitsYZ.GetX(), hitsYZ.GetY() ), axis=1 )
        # Test Everything by Drawing
        plot.rcParams['figure.max_open_warning'] = 100
        plot.figure(figsize=(3000*px,800*px))
        plot.scatter(arrayXZ[:,0],arrayXZ[:,1],s=1.4,marker=recMarker,c='black')
        plot.ylim(-800,800)
        plot.xlim(0,6000)
        plot.axis('off')
        imgbuf_XZ = io.BytesIO()
        plot.savefig(imgbuf_XZ, format='png', bbox_inches='tight') #for phi (0 to 360), estimate from horizontal
        im = Image.open(imgbuf_XZ)
        #im.show(title="XZ")

        print("Run %i Subrun %i Event %i, %s"%(run,subrun,event,ProcessTimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S.%f')))

        plot.figure(figsize=(3000*px,800*px))
        plot.scatter(arrayYZ[:,0],arrayYZ[:,1],s=1.4,marker=recMarker,c='black')
        plot.ylim(-800,800)
        plot.xlim(0,6000)
        plot.axis('off')
        imgbuf_YZ = io.BytesIO() #opening image buffer
        plot.savefig(imgbuf_YZ, format='png', bbox_inches='tight') #for theta (0 to 180), estimate from vertical
        im = Image.open(imgbuf_YZ)
        #im.show(title="YZ")

        imageXZ = Image.open(imgbuf_XZ)
        imageYZ = Image.open(imgbuf_YZ)
        occupyXZ = len([i for i in arrayXZ[:,0] if i>0])/172000
        occupyYZ = len([i for i in arrayYZ[:,0] if i>0])/172000
        occupancyXZ.append(occupyXZ)
        occupancyYZ.append(occupyYZ)
        asymXZ = shower.ImageAsym(imageXZ)
        AsymXZ.append(asymXZ)
        asymYZ = shower.ImageAsym(imageYZ)
        AsymYZ.append(asymYZ)

        print ("Asymmetry XZ = ", asymXZ)
        print ("Asymmetry YZ = ", asymYZ)
        print ("Occupancy XZ = ", occupyXZ)
        print ("Occupancy YZ = ", occupyYZ)

        AnglesDet = (shower.GetAnglesDet (imageXZ, imageYZ))
        print(AnglesDet)

        AngleXZ.append(AnglesDet[0])
        AngleYZ.append(AnglesDet[2])
        Evt_timestamp.append(timestamp)

    print(len(Run), len(Subrun), len(Event), len(S_start), len(S_stop), len(occupancyXZ), len(occupancyYZ), len(AsymXZ), len(AsymYZ), len(AngleXZ), len(AngleYZ), len(Evt_timestamp))
    for i in range(len(Run)):
        CombinedList.append((Run[i], Subrun[i], Event[i], S_start[i], S_stop[i], occupancyXZ[i], occupancyYZ[i], AsymXZ[i], AsymYZ[i], AngleXZ[i], AngleYZ[i], Evt_timestamp[i]))

    #print(len(AsymXZ), len(AsymYZ))
    if CombinedList and len(CombinedList) > 0:
        headers=['Run','Subrun','Event','Subrun_start','Subrun_stop', 'Oc_XZ', 'Oc_YZ','AsymXZ','AsymYZ','AngleXZ','AngleYZ','Timestamp'] #, 'OccupancyXZ', 'OccupancyYZ'
        formats=['%d','%d','%d','%d','%d','%.3f','%.3f','%.2f','%.2f','%.2f','%.2f','%d']
        output_file_name = f"{file_base_name}_EventInfo.txt"
        np.savetxt(output_file_name, CombinedList, delimiter='\t',fmt=formats, header='\t'.join(headers), comments='#')
    else:
        output_file_name = f"{file_base_name}_EventInfo.txt" 
        with open(output_file_name, "w"):
            pass

    plot.close('all') #closing all plots and canvas
    imgbuf_XZ.close() #closing image buffer
    imgbuf_YZ.close()

Welcome to JupyROOT 6.28/02
255
fardet_r00033429_s02_DDenergy_N23-07-05_v1_data.ultrashowerimages_ntuple
Run 33429 Subrun 12 Event 5325, 2019-06-23 21:11:52.118292
Asymmetry XZ =  0.24945928
Asymmetry YZ =  0.19187927
Occupancy XZ =  0.10034302325581396
Occupancy YZ =  0.14692441860465116
(91.20033342595165, 0.0, -2.9008057793831767, 0.0)
Run 33429 Subrun 28 Event 2106, 2019-06-23 21:52:12.952960
Asymmetry XZ =  0.22771627
Asymmetry YZ =  0.14964232
Occupancy XZ =  0.09375581395348838
Occupancy YZ =  0.17413953488372094
(0, 0, 0, 0)
Run 33429 Subrun 34 Event 7313, 2019-06-23 22:08:48.259868
Asymmetry XZ =  0.20325737
Asymmetry YZ =  0.15125959
Occupancy XZ =  0.09508720930232559
Occupancy YZ =  0.15390697674418605
(201.28091136426784, 0.3641066351960256, -21.631008613503752, 0.24831280752599763)
Run 33429 Subrun 35 Event 411, 2019-06-23 22:09:01.920839
Asymmetry XZ =  0.034941826
Asymmetry YZ =  0.02842965
Occupancy XZ =  0.21751162790697676
Occupancy YZ =  0.31983720930232556
(223.803

Asymmetry XZ =  0.042744838
Asymmetry YZ =  0.0365079
Occupancy XZ =  0.3012267441860465
Occupancy YZ =  0.3489127906976744
(73.29535982217281, 0.10002778549596769, 9.412614615170881, 0.16557544675212257)
Run 42783 Subrun 32 Event 4043, 2022-04-06 07:28:19.535973
Asymmetry XZ =  0.22379
Asymmetry YZ =  0.18792398
Occupancy XZ =  0.11326744186046511
Occupancy YZ =  0.1641220930232558
(42.08669074742984, 0.5207212481839741, 16.99972214504028, 0.1556870705712272)
Run 42783 Subrun 35 Event 3012, 2022-04-06 07:36:19.108097
Asymmetry XZ =  0.13405383
Asymmetry YZ =  0.12095199
Occupancy XZ =  0.10945348837209302
Occupancy YZ =  0.21136046511627907
(332.33731592108916, 0.3010819727471668, 18.488469019171987, 0.0850072716909086)
Run 42783 Subrun 39 Event 1475, 2022-04-06 07:47:05.017194
Asymmetry XZ =  0.33817384
Asymmetry YZ =  0.19169994
Occupancy XZ =  0.0781046511627907
Occupancy YZ =  0.15163372093023256
(271.1253125868297, 0.07502083912197577, 4.242845234787453, 0.1455758477069149)
Run 4